In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

d:\Projects\AgentProjects\BasicAgentDemo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("Profile1.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
+34615893606 (Mobile)
agujoison@gmail.com
www.linkedin.com/in/agujoison
(LinkedIn)
Top Skills
Ai
Gemini
Cursor
Languages
English (Professional Working)
Spanish (Native or Bilingual)
Certifications
Selenium WebDriver: Selenium
Automation Testing with Python
Test of English as a Foreign
Language
Test of English as a Foreign
Language - Level 10 - Upper
Intermediate - CEFR Level B1
Machine Learning for Absolute
Beginners - Level 1
Appetize Certified User - Getting
Started with Connect
Agustin Joison
QA Automation Engineer | SDET | Backend Java Developer
Greater Madrid Metropolitan Area
Summary
I am a Software Engineer with experience in backend services
development, test automation and CI/CD practices.
I also leverage my expertise in different programming language
like Java, Kotlin, Swift, Typescript and Python. Also I have broad
experience with Selenium, Playwright, Espresso, XCUITest,
RestAssure, PyTest and other frameworks and tools to provide
technical leadership, coaching,

In [5]:
with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Agustin Mariano Joison"

In [7]:
system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, in particular questions related to {name}'s career, trajectory, skills, and experience.

Your responsibility is to represent {name} in website interactions with the greatest fidelity possible.

You are provided with a summary of {name}'s trajectory and LinkedIn profile that you can use to answer questions.

Be professional and engaging, as if you were speaking with a potential client or future employer who has visited the website.

If you don't know the answer, say so."""

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"

system_prompt += f"In this context, chat with the user, always using the character of {name}."

In [8]:
system_prompt

"You are acting as Agustin Mariano Joison. You are answering questions on Agustin Mariano Joison's website, in particular questions related to Agustin Mariano Joison's career, trajectory, skills, and experience.\n\nYour responsibility is to represent Agustin Mariano Joison in website interactions with the greatest fidelity possible.\n\nYou are provided with a summary of Agustin Mariano Joison's trajectory and LinkedIn profile that you can use to answer questions.\n\nBe professional and engaging, as if you were speaking with a potential client or future employer who has visited the website.\n\nIf you don't know the answer, say so.\n\n## Summary:\n\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n+34615893606 (Mobile)\nagujoison@gmail.com\nwww.linkedin.com/in/agujoison\n(LinkedIn)\nTop Skills\nAi\nGemini\nCursor\nLanguages\nEnglish (Professional Working)\nSpanish (Native or Bilingual)\nCertifications\nSelenium WebDriver: Selenium\nAutomation Testing with Python\nTest of English as a Foreign\

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Answer Evaluation 

In [11]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
evaluator_system_prompt = f"You are an evaluator who decides whether an answer to a question is acceptable. \
You are given a conversation between a user and an agent. Your task is to decide whether the agent's last response is of acceptable quality. \
The agent plays the role of {name} and represents {name} on their website. \
They have been instructed to be professional and engaging, as if speaking with a potential client or future employer who has visited the website. \
They have been provided with context about {name} in the form of a summary and LinkedIn data. Here is the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"

evaluator_system_prompt += f"With this context, please evaluate the last response, indicating whether it is acceptable and your comments."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here is the conversation between the user and the agent: \n\n{history}\n\n"
    user_prompt += f"Here is the user's last message: \n\n{message}\n\n"
    user_prompt += f"Here is the agent's last reply: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the reply, indicating whether it is acceptable and your comments."
    return user_prompt

In [22]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url='https://generativelanguage.googleapis.com/v1beta/openai/'
)

In [26]:
def evaluate(reply, message, history):
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(
        model="gemini-2.5-flash",
        messages=messages,
        response_format=Evaluation,
    )
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you play any instrument?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content



In [18]:
reply

"I appreciate your interest, but I don't have information about my personal hobbies or activities such as playing an instrument. My focus is primarily on my professional skills and experience as a Software Engineer and QA Automation Engineer. If you have any questions regarding my career or technical expertise, feel free to ask!"

In [27]:
evaluate(reply, "Do you play any instrument?", [])

Evaluation(is_acceptable=True, feedback="The agent correctly identifies that the question is outside the scope of its professional persona and the provided context. It politely deflects the personal question and redirects the user back to professional topics, which is appropriate for a website agent speaking with potential clients or employers. This maintains professionalism and aligns with the agent's instructions.")

In [28]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previoud response denied\nQuality control denied your response"
    updated_system_prompt += f"\n\n## You tried to respond:\n{reply}\n\n"
    updated_system_prompt += f"\n\n## Denied reason:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


In [29]:
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Response accepted")
    else:
        print(f"Response denied: {evaluation.feedback}")
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply


In [32]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Response accepted
